In [2]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime

In [52]:
ee.Initialize()

# VETTORI
countries = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')
wsheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

In [53]:
# AREA OF INTEREST
region = [[-25.0, -40.0], [65.0, -40.0], [65.0, 40.0], [-30.0, 40.0], [-30.0, -40.0]]

In [54]:
wpbm_stats = ee.Image('projects/fao-wapor/L1-WPbmY2015-sample');

In [55]:
def vizualize_image(wbpm,regione_plot,scala):    
    visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region": regione_plot,
             "scale" : scala}
    
    url_WPbm = wbpm.getThumbUrl(visparams)
    return url_WPbm

In [56]:
%matplotlib inline  

In [57]:
Image(url=vizualize_image(wpbm_stats,region,250))


# Statistiche WPbm per paese e anno

In [58]:
africa_bbox = ee.Geometry.Rectangle(-15.64, -33.58, 59.06, 25.96)
filtered = countries.filterBounds(africa_bbox)

In [59]:
largeSheds = wsheds.filter(ee.Filter.gt('AreaSqKm', 25000))
'Count after filtering by size:', largeSheds.size().getInfo()

('Count after filtering by size:', 167)

In [66]:
nomeCountry = 'Sudan'
justCountry = countries.filter(ee.Filter.eq('Country', nomeCountry))
cutPoly = justCountry.geometry() 
cutBoundingBox = cutPoly.bounds(1)

country_mean = wpbm_stats.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = cutPoly,
    scale = 250,
    maxPixels = 1e9
)
mean = country_mean.getInfo()
mean['mean'] = mean.pop('b1')

reducers = ee.Reducer.minMax().combine(
  reducer2 = ee.Reducer.stdDev(),
  sharedInputs = True
)

# Use the combined reducer to get the min max and SD of the image.
stats = wpbm_stats.reduceRegion(
  reducer = reducers,
  bestEffort = True,
  geometry = cutPoly,
  scale = 250,
)

# Display the dictionary of band means and SDs.
min_max_std = stats.getInfo()
min_max_std['min'] = min_max_std.pop('b1_min')
min_max_std['std'] = min_max_std.pop('b1_stdDev')
min_max_std['max'] = min_max_std.pop('b1_max')
min_max_std

{'max': 3.7759787530379403, 'min': 0.0, 'std': 0.23828625422463104}

In [67]:
means = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = 250,
  );
means_data_by_country = means.getInfo()

In [68]:
means_data_by_country['columns']

{u'Country': u'String', u'mean': u'Float'}

In [69]:
means_data_by_country['features'][12]['properties']

{u'Country': u'Sudan', u'mean': 0.3441472837191009}

In [70]:
numero_record = means.size().getInfo()

In [71]:
for i in range(0,numero_record):
    print(means_data_by_country['features'][i]['properties'])

{u'Country': u'Rwanda', u'mean': 1.164326739511026}
{u'Country': u'Algeria', u'mean': 0.6808311929594619}
{u'Country': u'Sao Tome and Principe', u'mean': 1.8499667638270114}
{u'Country': u'Saudi Arabia', u'mean': 0.08668989420929085}
{u'Country': u'Senegal', u'mean': 0.2662467370405937}
{u'Country': u'Angola', u'mean': 0.9876436615908387}
{u'Country': u'Seychelles', u'mean': 1.2708032507791445}
{u'Country': u'Sierra Leone', u'mean': 0.7396157895931031}
{u'Country': u'Somalia', u'mean': 0.21821301853499758}
{u'Country': u'South Africa', u'mean': 0.7950867834063201}
{u'Country': u'Bahrain', u'mean': 0.06349408950078336}
{u'Country': u'Benin', u'mean': 0.4013888918232613}
{u'Country': u'Sudan', u'mean': 0.3441472837191009}
{u'Country': u'Swaziland', u'mean': 0.8651705265546427}
{u'Country': u'Botswana', u'mean': 0.5291074103279041}
{u'Country': u'Burkina Faso', u'mean': 0.2569482956833182}
{u'Country': u'Tanzania, United Republic of', u'mean': 1.0916336193920029}
{u'Country': u'Burundi', 